## This notebook is used to test if not using retrain_and_retrieve can generate same results

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import sys
current_file_path = Path.cwd().resolve()
root_path = current_file_path.parent
sys.path.append(str(root_path))

from sklearn.metrics import mean_squared_error
from configs.config_common import get_common_config
from views_forecasts.extensions import *

In [2]:
pd.read_parquet(root_path/"data/generated/generated.parquet")

step_pred_1  step_pred_2  step_pred_3  step_pred_4  \
month_id priogrid_gid                                                       
529      62356            0.022555     0.021456     0.021772     0.023067   
         79599            0.022555     0.021456     0.021772     0.023067   
         79600            0.022555     0.021456     0.021772     0.023067   
         79601            0.022555     0.021456     0.021772     0.023067   
         80317            0.022555     0.021456     0.036531     0.023067   
...                            ...          ...          ...          ...   
565      190496           0.022555     0.021456     0.021772     0.023067   
         190507           0.022555     0.021456     0.021772     0.023067   
         190508           0.022555     0.021456     0.021772     0.023067   
         190510           0.022555     0.021456     0.021772     0.023067   
         190511           0.022555     0.021456     0.021772     0.023067   

                       step_pred_5  step_pred_6  step_pred_7  step_pred_8  \
month_id priogrid_gid                                                       
529      62356            0.022615     0.021842     0.024531     0.024614   
         79599            0.022615     0.021842     0.024531     0.024614   
         79600            0.022615     0.021842     0.024531     0.024614   
         79601            0.022615     0.021842     0.024531     0.024614   
         80317            0.022615     0.021842     0.024531     0.024614   
...                            ...          ...          ...          ...   
565      190496           0.022615     0.021842     0.024531     0.024614   
         190507           0.022615     0.021842     0.024531     0.024614   
         190508           0.022615     0.021842     0.024531     0.024614   
         190510           0.022615     0.021842     0.024531     0.024614   
         190511           0.022615     0.021842     0.024531     0.024614   

                       step_pred_9  step_pred_10  ...  step_pred_27  \
month_id priogrid_gid                             ...                 
529      62356            0.025675      0.027789  ...      0.017809   
         79599            0.025675      0.027789  ...      0.017809   
         79600            0.025675      0.027789  ...      0.017809   
         79601            0.025675      0.027789  ...      0.017809   
         80317            0.025675      0.027789  ...      0.041920   
...                            ...           ...  ...           ...   
565      190496           0.025675      0.027789  ...      0.017809   
         190507           0.025675      0.027789  ...      0.017809   
         190508           0.025675      0.027789  ...      0.017809   
         190510           0.025675      0.027789  ...      0.017809   
         190511           0.025675      0.027789  ...      0.017809   

                       step_pred_28  step_pred_29  step_pred_30  step_pred_31  \
month_id priogrid_gid                                                           
529      62356             0.015909      0.016727      0.018867      0.017129   
         79599             0.015909      0.016727      0.018867      0.017129   
         79600             0.015909      0.016727      0.018867      0.017129   
         79601             0.015909      0.016727      0.018867      0.017129   
         80317             0.065944      0.076571      0.053233      0.062073   
...                             ...           ...           ...           ...   
565      190496            0.015909      0.016727      0.018867      0.017129   
         190507            0.015909      0.016727      0.018867      0.017129   
         190508            0.015909      0.016727      0.018867      0.017129   
         190510            0.015909      0.016727      0.018867      0.017129   
         190511            0.015909      0.016727      0.018867      0.017129   

                       step_pred_32  step

In [6]:
from ingester3.ViewsMonth import ViewsMonth
ViewsMonth.now()

ViewsMonth(531)

In [5]:
ViewsMonth.end_date()

TypeError: 'property' object is not callable

In [16]:
def evaluate_model(config):
    print('Evaluating...')
    steps = config['steps']
    stepcols = [config['depvar']]
    for step in steps:
        stepcols.append('step_pred_' + str(step))

    df = pd.DataFrame.forecasts.read_store(name=config["name"]).replace([np.inf, -np.inf], 0)[stepcols]

    pred_cols = [f'step_pred_{str(i)}' for i in steps]
    df['mse'] = df.apply(lambda row: mean_squared_error([row['ged_sb_dep']] * 36,
                                                        [row[col] for col in pred_cols]), axis=1)
    print(df['mse'].mean())

In [17]:
common_config = get_common_config()
evaluate_model(common_config)

Evaluating...
pr_1_orange_pasta.parquet
109.74281157762799


In [18]:
# note that read_store were commented out in the evaluate_model function when doing this 
%run ../main.py

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xiaolong-sun (views_pipeline). Use `wandb login --relogin` to force relogin


Training...
Evaluating...
pr_1_orange_pasta.parquet
109.74281157762799
Predicting...


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
